# Home work "CTC-loss"

Задача: реализовать прямой (forward) проход и вчислить матрицу $\alpha_t(s)$. Значения этой матрицы должны равняться
- $\alpha_t(s)$ в случае если валидные пути проходят через данную ячейку
- 0.0 в противном случае

Размерности входов и результата см. по коду.

Ноутбук с решением присыслать на voropaev@corp.mail.ru Тему письма пишите пожалуйста в формате "[МИФИ] Фамилия"

Deadline: 21.12.2018

In [256]:
import torch

In [258]:
# aap-ple- > apple
# удобные значения для отладки
l = torch.tensor([1,2,2,3,4,], dtype=torch.long)
y = torch.tensor(
[[0.6,  0.,  0., 0., 0.4,],    # aaa--
 [0.7, 0.1, 0., 0., 0.2,],     # aaaaaa--p
 [0.4, 0.6, 0., 0., 0.,],      # aappp
 [0., 0.4, 0., 0., .6,],       # ---pp
 [0., 0.6, 0.4, 0., 0.,],      # pppll
 [0,  0., 0.6, 0.4, 0.,],      # lllee
 [0., 0., 0., 0.6, 0.4,],      # eee--
 [0., 0., 0., 0.2, 0.8,],],    # ----e
 dtype=torch.float32)

# Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,2,3,1,4,5,4,3,2,1,], dtype=torch.long)

# y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1,  0,  0., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0,  0., 1., 0., 0., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 0., 0., 1.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],], dtype=torch.float32)

In [308]:
# Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,3,4,4,4,5,], dtype=torch.long)

# y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1,  1,  1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1,  1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],
 [1., 1., 1., 1., 1., 1.],], dtype=torch.float32)

In [309]:
def compute_ctc_alpha(l, y):
    """
    Функция, вычисляющая матрицу $\alpha$ для данного входа.
    @param l метки символов строки, размерностью [L,]
    @param y предсказанные сетью вероятности для каждого фрейма. Размерность [T, Lexicon+1]
    @return матрицу $\alpha$ размерностью [2*L+1, T]
    """
    s_max = 2 * l.size(0) + 1
    t_min = l.size(0)
    t_max = y.size(0)
    
    l_prime = sum([[i, 0] for i in l], [0]) # [1,2,2,3,4] -> [0,1,0,2,0,2,0,3,0,4,0]
    padding = 0
    
    # маска для обработки повторений символов
    # [0,1,0,2,0,2,0,3,0,4,0] -> [0,0,0,0,1,1,0,0,0,0,0]
    mask = [0] * s_max
    previous = None
    for n in range(1, s_max, 2):
        current = l_prime[n].item()
        if current == previous:
            mask[n] = 1
            mask[n-1] = 1
        else:
            mask[n-1] = 0
        previous = current
    
    print(mask)
    
    alpha = torch.zeros(s_max, t_max)
    alpha[0,0] = y[0, -1]
    alpha[1,0] = y[0, 0]
    
    visited = torch.zeros(s_max, t_max)
    visited[:,0] = 1
    
    def a(s, t):
        nonlocal padding
        border = s_max - 2 * (t_max - t - 1) - padding
        if s < 0 or s < border:
            return 0
        if visited[s, t]:
            return alpha[s, t]
        visited[s, t] = 1
        
        sum_ = a(s, t-1) + a(s-1, t-1)
        
        current = l_prime[s] - 1
        previous = (l_prime[s-2] - 1) if (s > 1) else current
        if (current != previous and current != -1):
            sum_ += a(s-2, t-1)
        return y[t, current] * sum_
    
    for t in range(1, t_max):
        padding += mask[t]
        for s in range(s_max):
            alpha[s, t] = a(s, t)
            print(alpha)
    return alpha

In [310]:
# Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha
def test():
    al = compute_ctc_alpha(l, y)
    right_al = torch.tensor([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    
    if torch.all(al == right_al):
        return True
    else:
        return False
    
assert test(), "Test faled"
    

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
   

RuntimeError: The size of tensor a (10) must match the size of tensor b (14) at non-singleton dimension 1